In [14]:
# Treasury Forecasting algo version #2 determining the best investment windows
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

# load variables from a .env file into the environment
load_dotenv(verbose=False)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [15]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1', sql=False):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        if sql:
            cursor.execute(sql)
        else:
            query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
            cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [16]:
# STEP 1: Get the asset classes combined with their parent class ID
table_name = 'AssetClass'
sql = """SELECT a.`ID`, a.`Title`, a.`Group`, a.`Issuer`, a.`PercentMax`,
CASE WHEN p.`Title` IS NULL THEN a.`Title` ELSE p.`Title` END AS `AssetClassCombined`
FROM `AssetClass` a
LEFT JOIN ( SELECT ac.`ID`, ac.`Title`, ac.`Group`, ac.`PercentMax` FROM `AssetClass` ac
WHERE AssetClassParentID = 0 ) p ON p.ID = a.AssetClassParentID
WHERE AssetClassParentID = 0 AND a.Title != 'Not Assigned'
"""
asset_classes = fetch_data(table_name, '',1,sql)
asset_classes

,ID,Title,Group,Issuer,PercentMax,AssetClassCombined
0,1,Cash/Sweep,Not Assigned,None,1.0000,Cash/Sweep
1,2,Certificate of Deposit,Certificates of Deposit,None,0.5000,Certificate of Deposit
2,3,Commercial Paper,Commercial Paper,None,0.3330,Commercial Paper
3,4,US Agencies,US Agencies,None,0.2000,US Agencies
4,7,Money Market,Not Assigned,None,0.2000,Money Market
5,8,Mutual Fund,Not Assigned,None,0.2000,Mutual Fund
6,16,US Treasuries,US Treasuries,None,1.0000,US Treasuries


In [ ]:
# STEP 4: Running balance day view taken from the SQL views
# TODO: replace the SQL view with pandas processign

table_name = 'RunningBalanceDayView'
running_balances = fetch_data(table_name)


In [18]:
# Add the daily total portfolio balance to the running balances DataFrame

# Convert TransactionDate to datetime if not already
running_balances['TransactionDate'] = pd.to_datetime(running_balances['TransactionDate'])

# Filter and pivot Portfolio balances
portfolio_balances = running_balances[running_balances['TransactionClass'] == 'Portfolio'].copy()
portfolio_balances['RunningTotal'] = pd.to_numeric(portfolio_balances['RunningTotal'])

# Create a series with daily portfolio balances
daily_portfolio = portfolio_balances.set_index('TransactionDate')['RunningTotal']

# Add portfolio balance to runningBalances DataFrame
running_balances = running_balances.merge(
    daily_portfolio.reset_index().rename(columns={'RunningTotal': 'Portfolio'}),
    on='TransactionDate',
    how='left'
)

In [19]:
# Add the asset class PercentMax column to the running balances DataFrame matching the on TransactionClass column

# Create a mapping dictionary from Title to PercentMax
percentmax_mapping = dict(zip(asset_classes['Title'], asset_classes['PercentMax']))

# Update PercentMax in running_balances using the mapping
running_balances['PercentMax'] = running_balances['TransactionClass'].map(percentmax_mapping).fillna(1.0)

# Convert PercentMax to float
# running_balances['PercentMax'] = pd.to_numeric(running_balances['PercentMax'])

# Check the PercentMax mapping
# percentmax_mapping
# running_balances #.info()

In [20]:
# Compute the asset class's maximum based on the policy
#       NOTE: The PercentMax map() inserts an object instead of a float. The object->float conversion resets the value to 0.0. TODO: Fix this.
                    # running_balances['PolicyMax'] = 0
running_balances['PolicyMax'] = running_balances['Portfolio'] * running_balances['PercentMax'].astype(float)
# running_balances.info()

In [21]:
# Add the daily available cash balance to the running balances DataFrame

# Filter and pivot cash balances
cash_balances = running_balances[running_balances['TransactionClass'] == 'Cash/Sweep'].copy()
cash_balances['RunningTotal'] = pd.to_numeric(cash_balances['RunningTotal'])

# Create a series with daily portfolio balances
daily_cash = cash_balances.set_index('TransactionDate')['RunningTotal']

# Add portfolio balance to runningBalances DataFrame
running_balances = running_balances.merge(
    daily_cash.reset_index().rename(columns={'RunningTotal': 'CashSweep'}),
    on='TransactionDate',
    how='left'
)

In [ ]:
# Add the investable column
#       if [TransactionClass] <> "Portfolio" && [TransactionClass] <> "Cash/Sweep",
#       then [PolicyMax]-[RunningTotal] else [RunningTotal]

# Convert RunningTotal to numeric if it's not already
running_balances['RunningTotal'] = pd.to_numeric(running_balances['RunningTotal'])

# Add the Investable column based on the condition
running_balances['Investable'] = np.where(
    (running_balances['TransactionClass'] != 'Portfolio') & (running_balances['TransactionClass'] != 'Cash/Sweep'),
    # investable can't fall below 0
    np.maximum(0, running_balances['PolicyMax'] - running_balances['RunningTotal']),
    np.maximum(0, running_balances['RunningTotal'])
)


In [ ]:
# Add the final available column
#       if ([TransactionClass] <> "Portfolio" && [TransactionClass] <> "Cash/Sweep",
#       then MIN([CashSweep],[Investable]) else [RunningTotal])
running_balances['Available'] = np.where(
    (running_balances['TransactionClass'] != 'Portfolio') &
    (running_balances['TransactionClass'] != 'Cash/Sweep'),
    # available can't fall below 0
    np.maximum(0, np.minimum(running_balances['CashSweep'], running_balances['Investable'])),
    np.maximum(0, running_balances['RunningTotal'])
)

In [52]:
# Test output the final dataframe

# Suppress scientific notation by setting float_format
pd.options.display.float_format = '{:,.0f}'.format

# Display the dataframe without scientific notation
running_balances[running_balances['TransactionClass'] == 'US Agencies']


,TransactionDate,TransactionClass,RunningTotal,Portfolio,PercentMax,PolicyMax,CashSweep,Investable,Available
2934,2025-06-03,US Agencies,"110,436,661","552,210,848",0.2000,"110,442,170","12,934,603","5,508","5,508"
2935,2025-06-04,US Agencies,"117,576,661","604,312,848",0.2000,"120,862,570","14,904,603","3,285,908","3,285,908"
2936,2025-06-05,US Agencies,"116,576,661","600,470,848",0.2000,"120,094,170","38,554,603","3,517,508","3,517,508"
2937,2025-06-06,US Agencies,"109,576,661","493,170,848",0.2000,"98,634,170","-57,745,397",0,0
2938,2025-06-07,US Agencies,"109,576,661","493,170,848",0.2000,"98,634,170","-57,745,397",0,0
...,...,...,...,...,...,...,...,...,...
3418,2026-09-30,US Agencies,"6,216,661","-25,184,796",0.2000,"-5,036,959","-112,197,397",0,0
3419,2026-10-01,US Agencies,"6,216,661","-25,184,796",0.2000,"-5,036,959","-112,197,397",0,0
3420,2026-10-02,US Agencies,"6,216,661","-25,184,796",0.2000,"-5,036,959","-112,197,397",0,0
3421,2026-10-03,US Agencies,"6,216,661","-25,184,796",0.2000,"-5,036,959","-112,197,397",0,0


In [54]:
# Save the final DataFrame to a pickle file
running_balances.to_pickle('running_balances.pkl')

In [55]:
# running_balances.head(5)
running_balances[running_balances['TransactionDate'] <= '2025-06-09']


,TransactionDate,TransactionClass,RunningTotal,Portfolio,PercentMax,PolicyMax,CashSweep,Investable,Available
0,2025-06-03,Portfolio,"552,210,848","552,210,848",1,"552,210,848","12,934,603","552,210,848","552,210,848"
1,2025-06-04,Portfolio,"604,312,848","604,312,848",1,"604,312,848","14,904,603","604,312,848","604,312,848"
2,2025-06-05,Portfolio,"600,470,848","600,470,848",1,"600,470,848","38,554,603","600,470,848","600,470,848"
3,2025-06-06,Portfolio,"493,170,848","493,170,848",1,"493,170,848","-57,745,397","493,170,848","493,170,848"
4,2025-06-07,Portfolio,"493,170,848","493,170,848",1,"493,170,848","-57,745,397","493,170,848","493,170,848"
5,2025-06-08,Portfolio,"493,170,848","493,170,848",1,"493,170,848","-57,745,397","493,170,848","493,170,848"
6,2025-06-09,Portfolio,"486,770,848","486,770,848",1,"486,770,848","-57,745,397","486,770,848","486,770,848"
489,2025-06-03,Cash/Sweep,"12,934,603","552,210,848",1.0000,"552,210,848","12,934,603","12,934,603","12,934,603"
490,2025-06-04,Cash/Sweep,"14,904,603","604,312,848",1.0000,"604,312,848","14,904,603","14,904,603","14,904,603"
491,2025-06-05,Cash/Sweep,"38,554,603","600,470,848",1.0000,"600,470,848","38,554,603","38,554,603","38,554,603"
